In [ ]:
# we need the following libraries, so let's install them
%pip install matplotlib
%pip install scikit-learn
%pip install tensorflow
%pip install imblearn

In [ ]:
# importing libraries
from time import time
import multiprocessing
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from glob import glob
import skimage
from  imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import sys
import warnings
from sklearn.utils import shuffle
import random
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder,OneHotEncoder
import logging
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore", category=DeprecationWarning)
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:

# loadnig the data and removing the useless column 'Unnamed: 0'
whole_data = pd.read_csv('/content/gdrive/MyDrive/radiomics/MIMIC_synced_data_nan_free_.csv')
# whole_data = pd.read_csv('Chexpert_synced_data_nan_free.csv')
whole_data.head(5)

,Unnamed: 0,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,0,-872.712177,-30.834509,1.183500e+10,5.213223,484.245094,2.123297,127.427505,259.288179,-535.029395,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,-860.588226,-94.596062,1.748392e+10,5.468412,406.884354,2.973349,455.105927,238.126282,-510.922325,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2,-771.608655,246.313766,1.054451e+10,5.639891,569.798054,2.284384,601.678467,308.126190,-357.526186,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3,-918.119244,197.538876,1.041507e+10,5.650920,757.435298,1.774919,477.192200,369.400809,-432.636913,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,4,-809.137268,336.588287,1.613116e+10,5.824157,562.000023,2.493775,658.329407,330.944854,-337.573971,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0


In [ ]:
2# excludiung Ages with Nans
temporary_labels = np.zeros((whole_data.shape[0],))
for i, val in enumerate(whole_data['Race'].values):
    if 'WHITE' in val:
        temporary_labels[i] = 0
    elif 'BLACK' in val or 'AFRICAN AMERICAN' in val:
        temporary_labels[i] = 1
    elif 'ASIAN' in val:
        temporary_labels[i] = 2
    else:
        temporary_labels[i] = np.nan

In [ ]:
whole_data['new_race'] = temporary_labels.tolist()

In [ ]:
new_target = []
for string_target in whole_data['Sex'].values:
  if 'M\nName' in string_target:
    new_target.append(1)
  elif 'F\nName' in string_target:
    new_target.append(0)
  else:
    new_target.append(np.nan)
whole_data['New_sex']= new_target

In [ ]:
whole_data = whole_data[~pd.isnull(whole_data['new_race'])]

In [ ]:
random_seed = 21202
del whole_data['Unnamed: 0']
whole_data.head()
whole_data = whole_data.sample(frac=1.0, random_state=random_seed)

In [ ]:
# finding total number of samples:
n_samples = whole_data.shape[0]
print("number of samples are: {}".format(n_samples))

number of samples are: 180604


In [ ]:
# names of all targets:
targets_names = list(whole_data.iloc[:,464:])
print(targets_names)

['index', 'path', 'subject_id', 'study_id', 'dicom_id', 'PerformedProcedureStepDescription', 'ViewPosition', 'ProcedureCodeSequence_CodeMeaning', 'ViewCodeSequence_CodeMeaning', 'Race', 'Ethnicity', 'Sex', 'Age', 'No Finding', 'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 'Lung Opacity', 'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices', 'new_race']


In [ ]:
# finding the number of unique subjects:
unique_subjects = pd.unique(whole_data['subject_id'])
print('The number of unique subjects are: {0}'.format(len(unique_subjects)))

The number of unique subjects are: 42654


In [ ]:
# preallocating the frequency of session of the subjects:
# this is written to make sure that we dont use a subject in both training and testing split.
test_split = 0.06       # precent out of total samples
valid_split = 0.1       # precent out of total samples (this is the sumation of both training and validation splits)

n_test = n_samples * test_split
n_whole_train = n_samples - n_test

frequencies = np.ones((unique_subjects.shape[0], 4)) * np.nan
for i, sbj in tqdm(enumerate(unique_subjects)):
  frequencies[i, 0] = sbj
  frequencies[i, 1] = (whole_data['subject_id'] == sbj).sum()
cumolative_sums_of_samples = np.cumsum(frequencies[:,1])
frequencies[:, 2] = cumolative_sums_of_samples
frequencies[:, 3] = cumolative_sums_of_samples/cumolative_sums_of_samples[~0]

42654it [00:17, 2460.68it/s]


In [ ]:
# visualizing the frequency  of subjects and images taken from each
vis_frequencies = pd.DataFrame(frequencies,columns=['subject_ID','n_of_sessions/images','cumolative_sessions','preccent_of_number_of_images'])
vis_frequencies.head(15)

,subject_ID,n_of_sessions/images,cumolative_sessions,preccent_of_number_of_images
0,17061176.0,2.0,2.0,0.000011
1,14108608.0,4.0,6.0,0.000033
2,17996251.0,17.0,23.0,0.000127
3,14522445.0,19.0,42.0,0.000233
4,13774104.0,1.0,43.0,0.000238
5,19797687.0,36.0,79.0,0.000437
6,19380289.0,2.0,81.0,0.000448
7,14410396.0,19.0,100.0,0.000554
8,16468805.0,5.0,105.0,0.000581
9,15087570.0,14.0,119.0,0.000659


In [ ]:
# separating the testing split without having any mutual_subjects:
testing_subjects_masking = vis_frequencies['preccent_of_number_of_images'] < test_split
testing_subjects = vis_frequencies[testing_subjects_masking]['subject_ID'].values.tolist()
whole_train_subjects = vis_frequencies[~testing_subjects_masking]['subject_ID'].values.tolist()

In [ ]:
# excluding irrelevant features
excluding2ds =  np.concatenate( (np.arange(32,41,dtype=int),
                                 np.arange(32,41,dtype=int)+ 116,
                                 np.arange(32,41,dtype=int)+116*2,
                                 np.arange(32,41,dtype=int)+116*3))

whole_data.drop(whole_data.columns[excluding2ds.tolist()],axis = 1,inplace=True)

In [ ]:
# separating test_data and whole_trainig_data
whole_data = whole_data.sample(frac=1.0, random_state=random_seed)
Test = whole_data[whole_data['subject_id'].isin(testing_subjects)]
X_test, y_test = Test.iloc[:,:464-36].values, Test['new_race'].values

# separating test_data and whole_trainig_data
Train = whole_data[whole_data['subject_id'].isin(whole_train_subjects)]

# the size of training/testing_split:
print('The size of the testing split is: {}'.format(Test.shape))
print('The size of the whole trainig (train + valid) split is: {}'.format(Train.shape))

The size of the testing split is: (10833, 456)
The size of the whole trainig (train + valid) split is: (169771, 456)


In [ ]:
# splitting the validation split:
X_train, X_val, y_train, y_val = train_test_split(Train.iloc[:,:464-36].values,
                                                  Train['new_race'].values,
                                                  test_size=valid_split,
                                                  random_state=random_seed
                                                  )

In [ ]:
print('The size of the validation split is: {}'.format(X_val.shape))
X_train.shape, X_val.shape, y_train.shape, y_val.shape

The size of the validation split is: (16978, 428)


((152793, 428), (16978, 428), (152793,), (16978,))

In [ ]:
sm = SMOTE(random_state=21202)
X_train, y_train = sm.fit_resample(X_train, y_train)

In [ ]:
y_train  = pd.get_dummies(pd.DataFrame(y_train.tolist(),columns=['column_name']),columns=['column_name']).values * 1
y_val  = pd.get_dummies(pd.DataFrame(y_val.tolist(),columns=['column_name']),columns=['column_name']).values * 1
y_test  = pd.get_dummies(pd.DataFrame(y_test.tolist(),columns=['column_name']),columns=['column_name']).values * 1

In [ ]:
def classifer_1(features_number: int) -> callable:
    model = keras.Sequential()
    model.add(keras.layers.Dense(units=200, activation="relu", input_dim=features_number))
    model.add(keras.layers.Dropout(0.0))
    model.add(keras.layers.Dense(units=200, activation="relu"))
    model.add(keras.layers.Dropout(0.0))
    model.add(keras.layers.Dense(units=3,activation='softmax'))
    model.summary()
    return model

def compiler_1(X_train_, Y_train_, X_valid_, Y_valid_,mdl_callback):
  initial_learning_rate = 0.0001
  epochs = 500
  batch_size = 512
  model = classifer_1(features_number=X_train_.shape[1])
  # lss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
  lss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
  metrics = ['accuracy',
                        tf.keras.metrics.AUC(curve='PR',name='PR-curve'),
                        tf.keras.metrics.AUC(curve='ROC',name='ROC-curve')]
  # metrics = metrics=[tf.keras.metrics.RootMeanSquaredError()]
  # metrics = [tf.keras.metrics.RootMeanSquaredError()]
  # lss = 'mean_absolute_error'
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=initial_learning_rate,
                                                beta_1=0.9,
                                                beta_2=0.999,
                                                epsilon=1e-08),
                                                loss=lss,
                metrics=metrics)
  model.fit(x=X_train_,
            y=Y_train_,
            epochs=epochs,
            initial_epoch=0,
            batch_size=batch_size,
            validation_data=(X_valid_, Y_valid_),
            verbose=2,
            shuffle=True,
            callbacks=[mdl_callback])
  return model

In [ ]:
# full Model

X_train_ = X_train.copy()
X_val_ = X_val.copy()
X_test_ = X_test.copy()

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/mimic_race_3categ_' + 'full'+ '_'
# saving_path = 'chexpert_race_3categ_' + 'full'+ '_'

belongings={'mean':mean_val,
            'feature_number':None,
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)
checkpoint_filepath = saving_path +'_mdl.h5'
model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               84200     
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               40200     
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 3)                 603       
                                                                 
Total params: 125,003
Trainable params: 125,003
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500

Epoch 1: val_loss improved from inf t

KeyboardInterrupt: ignored

In [ ]:
# each segments:

segment_number = 3

segment_names = ['segment_0_','segment_1_','segment_2_','segment_3_']

X_train_ = X_train.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_val_ = X_val.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_test_ = X_test.copy()[:, 107*segment_number:107*(segment_number + 1)]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/mimic_race_3categ_' + segment_names[segment_number]+ '_'
# saving_path = 'chexpert_race_3categ_segment_' + segment_names[segment_number]+ '_'


belongings={'mean':mean_val,
            'feature_number':segment_names[segment_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}
np.save(saving_path + 'belongings.npy',belongings)
checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_0_0 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 200)               21200     
                                                                 
 dropout_6 (Dropout)         (None, 200)               0         
                                                                 
 dense_10 (Dense)            (None, 200)               40200     
                                                                 
 dropout_7 (Dropout)         (None, 200)               0         
                                                                 
 dense_11 (Dense)            (None, 3)                 603       
                                                                 
Total params: 62,003
Trainable params: 62,003
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500

Epoch 1: val_loss improved from inf t

In [ ]:
# each segments:
segment_number = 1

segment_names = ['segment_0_','segment_1_','segment_2_','segment_3_']

X_train_ = X_train.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_val_ = X_val.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_test_ = X_test.copy()[:, 107*segment_number:107*(segment_number + 1)]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

# saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_segment_' + segment_names[segment_number]+ '_'
saving_path = 'chexpert_race_3categ_segment_' + segment_names[segment_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':segment_names[segment_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'
# checkpoint_filepath = '_mdl.h5'
model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_0_1 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 100)               10600     
                                                                 
 dropout_14 (Dropout)        (None, 100)               0         
                                                                 
 dense_22 (Dense)            (None, 300)               30300     
                                                                 
 dropout_15 (Dropout)        (None, 300)               0         
                                                                 
 dense_23 (Dense)            (None, 3)                 903       
                                                                 
Total params: 41803 (163.29 KB)
Trainable params: 41803 (163.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500

Epo

KeyboardInterrupt: 

In [ ]:
# each segments:
segment_number = 2

segment_names = ['segment_0_','segment_1_','segment_2_','segment_3_']

X_train_ = X_train.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_val_ = X_val.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_test_ = X_test.copy()[:, 107*segment_number:107*(segment_number + 1)]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

# saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_segment_' + segment_names[segment_number]+ '_'
saving_path = 'chexpert_race_3categ_segment_' + segment_names[segment_number]+ '_'


belongings={'mean':mean_val,
            'feature_number':segment_names[segment_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_0_2 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 100)               10600     
                                                                 
 dropout_16 (Dropout)        (None, 100)               0         
                                                                 
 dense_25 (Dense)            (None, 300)               30300     
                                                                 
 dropout_17 (Dropout)        (None, 300)               0         
                                                                 
 dense_26 (Dense)            (None, 3)                 903       
                                                                 
Total params: 41803 (163.29 KB)
Trainable params: 41803 (163.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500

Epo

KeyboardInterrupt: 

In [ ]:
# each segments:
segment_number = 3

segment_names = ['segment_0_','segment_1_','segment_2_','segment_3_']

X_train_ = X_train.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_val_ = X_val.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_test_ = X_test.copy()[:, 107*segment_number:107*(segment_number + 1)]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

# saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_segment_' + segment_names[segment_number]+ '_'

saving_path = 'chexpert_race_3categ_segment_' + segment_names[segment_number]+ '_'


belongings={'mean':mean_val,
            'feature_number':segment_names[segment_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_0_3 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 100)               10600     
                                                                 
 dropout_18 (Dropout)        (None, 100)               0         
                                                                 
 dense_28 (Dense)            (None, 300)               30300     
                                                                 
 dropout_19 (Dropout)        (None, 300)               0         
                                                                 
 dense_29 (Dense)            (None, 3)                 903       
                                                                 
Total params: 41803 (163.29 KB)
Trainable params: 41803 (163.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500

Epo

KeyboardInterrupt: 

In [ ]:
fos_features = (np.arange(0,18) + 0).tolist() + (np.arange(0,18) + 107).tolist() + (np.arange(0,18) + 107*2).tolist() + (np.arange(0,18) + 107*3).tolist()
_3d_features = (np.arange(18,32) + 0).tolist() + (np.arange(18,32) + 107).tolist() + (np.arange(18,32) + 107*2).tolist() + (np.arange(18,32) + 107*3).tolist()
glcm_features = (np.arange(32,56) + 0).tolist() + (np.arange(32,56) + 107).tolist() + (np.arange(32,56) + 107*2).tolist() + (np.arange(32,56) + 107*3).tolist()
glszm_features = (np.arange(56,72) + 0).tolist() + (np.arange(56,72)+ 107).tolist() + (np.arange(56,72) + 107*2).tolist() + (np.arange(56,72) + 107*3).tolist()
glrlm_features = (np.arange(72,88) + 0).tolist() + (np.arange(72,88)  + 107).tolist() + (np.arange(72,88)  + 107*2).tolist() + (np.arange(72,88)  + 107*3).tolist()
ngtdm_features = (np.arange(88,93) + 0).tolist() + (np.arange(88,93)  + 107).tolist() + (np.arange(88,93)  + 107*2).tolist() + (np.arange(88,93) + 107*3).tolist()
gldm_features = (np.arange(93,107) + 0).tolist() + (np.arange(93,107)  + 107).tolist() + (np.arange(93,107)  + 107*2).tolist() + (np.arange(93,107) + 107*3).tolist()

feature_names = ['fos_features', '_3d_features', 'glcm_features', 'glszm_features', 'glrlm_features', 'ngtdm_features', 'gldm_features']
feature_indexes = [fos_features, _3d_features, glcm_features, glszm_features, glrlm_features, ngtdm_features, gldm_features]

In [ ]:
feature_number = 0

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_0 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 1

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_1 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 2

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_2 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 3

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_3 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 4

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_4 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 5

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_5 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 6

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_6 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)